## Splitter Pattern

This notebook demonstrates the **Splitter** Enterprise Integration Pattern (EIP) using Rustic AI agents.

The Splitter pattern takes a single composite message (such as an order with multiple items) and splits it into multiple individual messages, each of which can be processed independently. This is useful for:

* Handling multi-part messages (e.g., line items in an order)
* Enabling parallel processing of each item
* Routing different parts to different subsystems or agents

### Pattern Flow

**1→n Message Flow:**

```
[ProbeAgent] --OrderRequest--> (order with multiple items)
                    ↓ SPLIT PHASE
    [SplitterAgent] applies JSONata/Tokenizer/Custom logic:
    ├── OrderItemRequest(item 1)
    ├── OrderItemRequest(item 2)
    └── OrderItemRequest(item n)
                    ↓ 
    [ItemProcessorAgent] processes each item individually
```

### Key Components

* **SplitterAgent**: Applies a dynamic splitting strategy using `JSONata`, `Tokenizer`, or custom logic
* **SplitterConfig**: Specifies how to extract each part and optionally format them differently using `FormatSelector`
* **Multi-Format Output**: Each split message can carry a different payload and message format (e.g., JSON, media, etc.)
* **Message Fan-Out**: The agent emits n messages, one per element from the original input

This example showcases how to use the Splitter pattern to break down composite data structures and orchestrate downstream processing workflows effectively.


In [1]:
from rustic_ai.core import Guild, GuildTopics, Priority
from rustic_ai.core.agents.eip.splitter_agent import (
    FixedFormatSelector,
    ListFormatSelector,
    ListSplitter,
    SplitterAgent,
    SplitterConf,
)

In [2]:
from rustic_ai.core.agents.eip.basic_wiring_agent import BasicWiringAgent
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent
from rustic_ai.core.guild.builders import AgentBuilder, GuildBuilder, RouteBuilder
from rustic_ai.core.utils.basic_class_utils import get_qualified_class_name
from rustic_ai.core.utils.jexpr import JExpr, JObj, JxScript

## Message Types

First, let's define the message models for our scatter-gather pattern. We have:
- **Request messages**: One input message with a list of items (A Collection)
- **Result messages**: One message per item.

Look at [test_splitter_agent.py](./rustic-ai/core/tests/agents/test_splitter_agent.py) for the messages.

In [3]:
from typing import Any, Dict, List, Optional
from pydantic import BaseModel

In [4]:
class PurchaseOrderRequest(BaseModel):
    order_id: str
    items: List[Dict[str, Any]]
    customer: str


class ItemOrderList(BaseModel):
    item1: Dict
    item2: Dict


class ItemProcessingResult(BaseModel):
    id: Optional[str]
    quantity: Optional[int]


### Agent Specifications

Now let's create the agent specifications that will be used in our guild.

In [5]:
import shortuuid

In [6]:
splitter_conf = SplitterConf(
    splitter=ListSplitter(field_name="items"),
    format_selector=FixedFormatSelector(strategy="fixed", fixed_format=get_qualified_class_name(ItemProcessingResult)),
)

splitter_agent = (
    AgentBuilder(SplitterAgent)
    .set_id("SplitterAgent")
    .set_name("Purchase Order Splitter")
    .set_description("Splits a PurchaseOrderRequest into multiple ItemRequest messages")
    .set_properties(splitter_conf)
    .add_additional_topic("purchase_orders")
    .listen_to_default_topic(False)
    .build_spec()
)

### Routing Rules - The Splitting Phase

A simple message routed to the splitter agent to start the splitting phase

In [7]:
splitter_results_route = (
    RouteBuilder(splitter_agent)
    .on_message_format(ItemProcessingResult)
    .set_destination_topics("item_processing_results")
    .set_route_times(-1)
    .build()
)

# Create the Guild

In [8]:
import os
from rustic_ai.core.guild.metastore import Metastore

In [9]:
splitter_guild_builder = (
    GuildBuilder(
        guild_id=f"splitter_guild{shortuuid.uuid()}",
        guild_name="SplitterGuild",
        guild_description="Demonstrates splitting of messages using SplitterAgent",
    )
    .add_agent_spec(splitter_agent)
    .add_route(splitter_results_route)
)

In [10]:
db = "sqlite:///splitter_demo.db"

if os.path.exists("splitter_demo.db"):
    os.remove("splitter_demo.db")

Metastore.initialize_engine(db)
Metastore.get_engine(db)
Metastore.create_db()

In [ ]:
guild = splitter_guild_builder.bootstrap(metastore_database_url=db, organization_id="myorg")  # Use SQLite for simplicity

{'id': 'splitter_guildA6Z5nRYKavYWGyvga82s3L', 'execution_engine': 'rustic_ai.core.guild.execution.sync.sync_exec_engine.SyncExecutionEngine', 'backend_class': 'EmbeddedMessagingBackend', 'organization_id': 'myorg', 'status': 'stopped', 'description': 'Demonstrates splitting of messages using SplitterAgent', 'name': 'SplitterGuild', 'backend_module': 'rustic_ai.core.messaging.backend.embedded_backend', 'backend_config': {}, 'dependency_map': {}}


# Probe Agent steup

In [13]:
import time

time.sleep(2)

# Create probe agent to monitor the entire flow
probe_spec = (
    AgentBuilder(ProbeAgent)
    .set_id("TestProbe")
    .set_name("Test Probe Agent")
    .set_description("Monitors the entire scatter-gather flow")
    .add_additional_topic("purchase_orders") # Initial request
    .add_additional_topic("item_processing_results")  # Final results
    .build_spec()
)

# Add probe agent to the bootstrapped guild
probe_agent: ProbeAgent = guild._add_local_agent(probe_spec)  # type: ignore

# Test

In [15]:
# Create test data for analysis
test_order = PurchaseOrderRequest(
    order_id="PO-12345",
    customer="ACME Corp",
    items=[{"id": "item-001", "quantity": 2}, {"id": "item-002", "quantity": 1}],
)


print(f"Sending PurchaseOrderRequest with ID: {test_order.order_id}")
print(f"Items: {test_order.items}")

# Send through probe agent to trigger the scatter-gather flow
probe_agent.publish_with_guild_route(topic="purchase_orders", payload=test_order)

Sending PurchaseOrderRequest with ID: PO-12345
Items: [{'id': 'item-001', 'quantity': 2}, {'id': 'item-002', 'quantity': 1}]


In [16]:
# Let's check the message history to see the complete flow
probe_agent.print_all_history()


For message at index 0 (9567356143617970176):
	(purchase_orders) -> [Purchase Order Splitter/SplitterAgent:split_and_send] -> (item_processing_results)

For message at index 1 (9567356143647330304):
	(purchase_orders) -> [Purchase Order Splitter/SplitterAgent:split_and_send] -> (item_processing_results)


In [17]:
probe_agent.get_messages()

[Message(sender=AgentTag(id='SplitterAgent', name='Purchase Order Splitter'), topics='item_processing_results', recipient_list=[], payload={'id': 'item-001', 'quantity': 2}, format='__main__.ItemProcessingResult', in_response_to=9567356143601197056, thread=[9567356143601197056], conversation_id=None, forward_header=None, routing_slip=RoutingSlip(steps=[RoutingRule(agent=AgentTag(id=None, name='Purchase Order Splitter'), agent_type=None, method_name=None, origin_filter=None, message_format='__main__.ItemProcessingResult', destination=RoutingDestination(topics='item_processing_results', recipient_list=[], priority=None), mark_forwarded=False, route_times=-2, transformer=None, agent_state_update=None, guild_state_update=None, process_status=None)]), message_history=[ProcessEntry(agent=AgentTag(id='SplitterAgent', name='Purchase Order Splitter'), origin=9567356143601197056, result=9567356143617970176, processor='split_and_send', from_topic='purchase_orders', to_topics=['item_processing_res

In [19]:
from rustic_ai.core.agents.system.models import StopGuildRequest


probe_agent.publish_with_guild_route(
    topic=GuildTopics.SYSTEM_TOPIC, payload=StopGuildRequest(guild_id=guild.id)
)  # Send the test request to start the flow

guild.shutdown()

In [20]:
spec = splitter_guild_builder.build_spec()
dict_dump = spec.model_dump()

In [21]:
dict_dump

{'id': 'splitter_guildA6Z5nRYKavYWGyvga82s3L',
 'name': 'SplitterGuild',
 'description': 'Demonstrates splitting of messages using SplitterAgent',
 'properties': {},
 'agents': [{'id': 'SplitterAgent',
   'name': 'Purchase Order Splitter',
   'description': 'Splits a PurchaseOrderRequest into multiple ItemRequest messages',
   'class_name': 'rustic_ai.core.agents.eip.splitter_agent.SplitterAgent',
   'additional_topics': ['purchase_orders'],
   'properties': {'splitter': {'split_type': 'list', 'field_name': 'items'},
    'format_selector': {'strategy': 'fixed',
     'fixed_format': '__main__.ItemProcessingResult'}},
   'listen_to_default_topic': False,
   'act_only_when_tagged': False,
   'predicates': {},
   'dependency_map': {},
   'resources': {'num_cpus': None, 'num_gpus': None, 'custom_resources': {}},
   'qos': {'timeout': None, 'retry_count': None, 'latency': None}}],
 'dependency_map': {},
 'routes': {'steps': [{'agent': {'id': None,
     'name': 'Purchase Order Splitter'},
   

In [22]:
import yaml
with open('/home/nihal/Projects/ai-platform/rustic-ai/examples/notebooks/eip/007_splitter.yaml', 'w') as f:
    yaml.dump(dict_dump, f)